In [8]:
%%writefile core.py
import numba as nb
import pandas as pd
import numpy as np
from loguru import logger
import numexpr as ne
import sympy
import scipy
from math import * 
logger.info(f"Pandas version: {pd.__version__}")
logger.info(f"Numpy version: {np.__version__}")
logger.info(f"Numba version: {ne.__version__}")
logger.info(f"NumExpr version: {nb.__version__}")
logger.info(f"SymPy version: {sympy.__version__}")
logger.info(f"SciPy version: {scipy.__version__}")

def softplus_np(n):
    x = np.random.uniform(50,150,size = n).astype(np.float32)
    safe = x < 100
    return np.log(1 + np.exp(x * safe)) * safe + x * (1-safe)

def softplus_vec_wrap(n):
    x = np.random.uniform(50,150,size = n).astype(np.float32)
    @nb.vectorize('float32(float32)',target = "parallel")
    def softplus_nb(x):
        safe = x < 100
        return log(1 + exp(x * safe)) * safe + x * (1-safe)
    result = softplus_nb(x)
    return result

softplus_jit = nb.jit(softplus_np)
softplus_njit = nb.njit(softplus_np)
softplus_fastmath_njit = nb.njit(fastmath=True,parallel=True)(softplus_np)


params = {"n":2000000}
softplus_np(2000000)
softplus_jit(2000000)
softplus_njit(2000000)
softplus_fastmath_njit(2000000)
softplus_vec_wrap(**params)

Writing core.py


In [7]:
params = {"n":2000000}
%time softplus_np(2000000)
%time softplus_jit(2000000)
%time softplus_njit(2000000)
%time softplus_fastmath_njit(2000000)
%time softplus_vec_wrap(**params)

CPU times: user 42.7 ms, sys: 7.71 ms, total: 50.5 ms
Wall time: 49.3 ms
CPU times: user 69.7 ms, sys: 4.09 ms, total: 73.8 ms
Wall time: 73.6 ms
CPU times: user 70.5 ms, sys: 0 ns, total: 70.5 ms
Wall time: 70.4 ms
CPU times: user 71.5 ms, sys: 147 µs, total: 71.7 ms
Wall time: 20.9 ms
CPU times: user 161 ms, sys: 0 ns, total: 161 ms
Wall time: 122 ms


array([ 54.144817, 109.143684,        inf, ...,  68.4414  , 107.868805,
       112.75654 ], dtype=float32)